# Quasi-criticality in the Cortex

In [ ]:
'''Setup notebook environment -q flag suppresses output, if you want to see it, remove the -q flag'''
%pip install -r requirements.txt -q
from utils.plotting_utils import power_law_plot
from utils.data_utils import load_data_csv

In [ ]:
pattern_one = load_data_csv("data/with_pattern.csv")
no_pattern = load_data_csv("data/new_run_func.csv")

In [ ]:
print(pattern_one.head())
print(pattern_one.describe())

In [ ]:
print(no_pattern.head())
print(no_pattern.describe())

In [ ]:
# For avalache sizes
power_law_plot(pattern_one['size'], data_type='size')

# For avalanche durations
power_law_plot(pattern_one['duration'], data_type='duration')

In [ ]:
power_law_plot(no_pattern['size'], data_type='size')
power_law_plot(no_pattern['duration'], data_type='duration')